In [1]:
import numpy as np
from keras.models import Model, Sequential
from keras.layers import Input, LSTM, Dense, RNN
from keras import layers

C:\Users\charlychiu\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [3]:
TRAINING_SIZE = 80000
DIGITS = 3
REVERSE = False
MAXLEN = DIGITS + 1 + DIGITS
chars = '0123456789- '
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

### one-hot encoding converter

In [4]:
class CharacterTable(object):
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
    
    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[i] for i in x)

In [5]:
ctable = CharacterTable(chars)

### one-hot encoding map

In [6]:
ctable.indices_char

{0: ' ',
 1: '-',
 2: '0',
 3: '1',
 4: '2',
 5: '3',
 6: '4',
 7: '5',
 8: '6',
 9: '7',
 10: '8',
 11: '9'}

## Data Generation

- substraction data: 80000
- total data: 80000
- length of the sequence: 7

In [7]:
questions_with_minus = []
expected_with_minus = []
seen_with_minus = set()
print('Generating data...')
while len(questions_with_minus) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    key = tuple(sorted((a, b)))
    if a < b:
        continue
    if key in seen_with_minus:
        continue
    seen_with_minus.add(key)
    q = '{}-{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a - b)
    ans += ' ' * (DIGITS + 1 - len(ans))
    if REVERSE:
        query = query[::-1]
    questions_with_minus.append(query)
    expected_with_minus.append(ans)
print('Total subtraction questions:', len(questions_with_minus))

Generating data...
Total subtraction questions: 80000


### subtraction data sample

In [8]:
print(questions_with_minus[:5], expected_with_minus[:5])

['28-7   ', '76-7   ', '31-3   ', '5-0    ', '3-2    '] ['21  ', '69  ', '28  ', '5   ', '1   ']


### combine addtion & subtraction data

In [9]:
questions = questions_with_minus
expected = expected_with_minus

## Processing

### transfer data to one-hot representation

In [10]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(expected), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

Vectorization...


### Split data into training, validation, testing

In [11]:
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# train_test_split
train_x = x[:20000]
train_y = y[:20000]
test_x = x[20000:]
test_y = y[20000:]

split_at = len(train_x) - len(train_x) // 10
(x_train, x_val) = train_x[:split_at], train_x[split_at:]
(y_train, y_val) = train_y[:split_at], train_y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

print('Testing Data:')
print(test_x.shape)
print(test_y.shape)

Training Data:
(18000, 7, 12)
(18000, 4, 12)
Validation Data:
(2000, 7, 12)
(2000, 4, 12)
Testing Data:
(60000, 7, 12)
(60000, 4, 12)


In [12]:
print("input: ", x_train[:3], '\n\n', "label: ", y_train[:3])

input:  [[[False False False  True False False False False False False False
   False]
  [False False  True False False False False False False False False
   False]
  [False False False False False False  True False False False False
   False]
  [False  True False False False False False False False False False
   False]
  [False False False  True False False False False False False False
   False]
  [False False False False False False False False False False  True
   False]
  [ True False False False False False False False False False False
   False]]

 [[False False False False False False False False False  True False
   False]
  [False False False False  True False False False False False False
   False]
  [False False False False False  True False False False False False
   False]
  [False  True False False False False False False False False False
   False]
  [False False False False False False False False  True False False
   False]
  [False False False  True False False Fal

## Build Model

In [13]:
def get_model():
    model = Sequential()
    model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
    model.add(layers.RepeatVector(DIGITS + 1))
    for _ in range(LAYERS):
        model.add(RNN(HIDDEN_SIZE, return_sequences=True))
    model.add(layers.TimeDistributed(layers.Dense(len(chars), activation='softmax')))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [14]:
print('Build model...')
model = get_model()
model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 4, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


### Training

In [15]:
for iteration in range(100):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 0
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 7s 382us/step - loss: 1.8921 - acc: 0.3239 - val_loss: 1.7450 - val_acc: 0.3546
Q 228-66  T 162  ☒ 210 
Q 800-42  T 758  ☒ 210 
Q 249-19  T 230  ☒ 210 
Q 578-75  T 503  ☒ 210 
Q 353-77  T 276  ☒ 110 
Q 56-12   T 44   ☒ 11  
Q 659-385 T 274  ☒ 210 
Q 804-791 T 13   ☒ 210 
Q 565-48  T 517  ☒ 210 
Q 303-224 T 79   ☒ 110 

--------------------------------------------------
Iteration 1
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 4s 204us/step - loss: 1.7216 - acc: 0.3553 - val_loss: 1.6943 - val_acc: 0.35781.72
Q 693-273 T 420  ☒ 25  
Q 195-88  T 107  ☒ 10  
Q 566-548 T 18   ☒ 11  
Q 694-55  T 639  ☒ 250 
Q 400-24  T 376  ☒ 10  
Q 812-338 T 474  ☒ 10  
Q 717-89  T 628  ☒ 10  
Q 73-53   T 20   ☒ 10  
Q 981-61  T 920  ☒ 290 
Q 72-16   T 56   ☒ 10  

---------------

18000/18000 [==============================] - 4s 206us/step - loss: 1.1763 - acc: 0.5671 - val_loss: 1.1701 - val_acc: 0.5591
Q 892-7   T 885  ☒ 981 
Q 738-53  T 685  ☒ 711 
Q 554-217 T 337  ☒ 301 
Q 840-161 T 679  ☒ 789 
Q 422-38  T 384  ☒ 383 
Q 901-205 T 696  ☒ 761 
Q 273-104 T 169  ☒ 138 
Q 230-34  T 196  ☒ 153 
Q 638-32  T 606  ☒ 525 
Q 893-16  T 877  ☒ 871 

--------------------------------------------------
Iteration 15
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 4s 198us/step - loss: 1.1402 - acc: 0.5818 - val_loss: 1.1298 - val_acc: 0.5859
Q 752-91  T 661  ☒ 667 
Q 999-32  T 967  ☒ 965 
Q 350-91  T 259  ☒ 255 
Q 829-260 T 569  ☒ 588 
Q 505-13  T 492  ☒ 598 
Q 150-24  T 126  ☒ 115 
Q 90-51   T 39   ☒ 28  
Q 965-41  T 924  ☒ 931 
Q 60-15   T 45   ☒ 42  
Q 991-911 T 80   ☒ 19  

--------------------------------------------------
Iteration 16
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 4s 201us/step - loss: 0.8419 - acc: 0.6922 - val_loss: 0.8651 - val_acc: 0.6754
Q 579-283 T 296  ☒ 219 
Q 428-27  T 401  ☒ 403 
Q 99-83   T 16   ☒ 20  
Q 338-288 T 50   ☒ 12  
Q 619-50  T 569  ☒ 576 
Q 572-360 T 212  ☒ 217 
Q 969-875 T 94   ☒ 17  
Q 579-4   T 575  ☒ 573 
Q 576-64  T 512  ☑ 512 
Q 513-98  T 415  ☒ 416 

--------------------------------------------------
Iteration 29
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 4s 200us/step - loss: 0.8209 - acc: 0.7012 - val_loss: 0.8506 - val_acc: 0.6826
Q 284-67  T 217  ☒ 225 
Q 376-182 T 194  ☒ 108 
Q 904-572 T 332  ☒ 320 
Q 876-158 T 718  ☒ 717 
Q 170-47  T 123  ☒ 127 
Q 542-29  T 513  ☒ 515 
Q 435-20  T 415  ☒ 411 
Q 855-144 T 711  ☒ 719 
Q 958-872 T 86   ☒ 17  
Q 589-230 T 359  ☒ 377 

--------------------------------------------------
Iteration 30
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 4s 204us/step - loss: 0.4196 - acc: 0.8612 - val_loss: 0.4356 - val_acc: 0.8492
Q 512-66  T 446  ☑ 446 
Q 522-331 T 191  ☒ 190 
Q 612-2   T 610  ☑ 610 
Q 497-17  T 480  ☑ 480 
Q 972-458 T 514  ☒ 524 
Q 860-82  T 778  ☒ 779 
Q 578-3   T 575  ☑ 575 
Q 886-645 T 241  ☑ 241 
Q 982-288 T 694  ☒ 693 
Q 463-40  T 423  ☒ 422 

--------------------------------------------------
Iteration 43
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 4s 205us/step - loss: 0.3861 - acc: 0.8742 - val_loss: 0.4047 - val_acc: 0.8594
Q 834-39  T 795  ☒ 794 
Q 786-3   T 783  ☑ 783 
Q 945-842 T 103  ☒ 11  
Q 431-0   T 431  ☑ 431 
Q 402-40  T 362  ☑ 362 
Q 52-23   T 29   ☒ 39  
Q 717-677 T 40   ☒ 39  
Q 370-69  T 301  ☑ 301 
Q 591-8   T 583  ☑ 583 
Q 430-60  T 370  ☒ 379 

--------------------------------------------------
Iteration 44
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 4s 209us/step - loss: 0.1502 - acc: 0.9621 - val_loss: 0.1854 - val_acc: 0.9424
Q 354-61  T 293  ☑ 293 
Q 888-99  T 789  ☑ 789 
Q 244-37  T 207  ☑ 207 
Q 746-372 T 374  ☑ 374 
Q 372-84  T 288  ☑ 288 
Q 184-49  T 135  ☑ 135 
Q 422-15  T 407  ☑ 407 
Q 722-131 T 591  ☑ 591 
Q 705-15  T 690  ☒ 680 
Q 98-40   T 58   ☑ 58  

--------------------------------------------------
Iteration 57
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 4s 206us/step - loss: 0.1419 - acc: 0.9647 - val_loss: 0.1643 - val_acc: 0.9521
Q 930-228 T 702  ☒ 602 
Q 634-235 T 399  ☑ 399 
Q 796-15  T 781  ☑ 781 
Q 283-96  T 187  ☑ 187 
Q 329-47  T 282  ☑ 282 
Q 429-67  T 362  ☑ 362 
Q 453-353 T 100  ☑ 100 
Q 694-55  T 639  ☑ 639 
Q 795-656 T 139  ☑ 139 
Q 439-204 T 235  ☒ 234 

--------------------------------------------------
Iteration 58
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 4s 217us/step - loss: 0.0892 - acc: 0.9758 - val_loss: 0.1216 - val_acc: 0.9594
Q 721-76  T 645  ☑ 645 
Q 634-235 T 399  ☑ 399 
Q 733-348 T 385  ☑ 385 
Q 693-273 T 420  ☑ 420 
Q 88-51   T 37   ☑ 37  
Q 441-6   T 435  ☑ 435 
Q 350-91  T 259  ☑ 259 
Q 108-46  T 62   ☑ 62  
Q 946-657 T 289  ☒ 299 
Q 356-90  T 266  ☑ 266 

--------------------------------------------------
Iteration 71
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 4s 221us/step - loss: 0.0744 - acc: 0.9823 - val_loss: 0.0966 - val_acc: 0.9690
Q 190-31  T 159  ☒ 169 
Q 420-3   T 417  ☒ 407 
Q 945-30  T 915  ☑ 915 
Q 70-68   T 2    ☑ 2   
Q 836-26  T 810  ☑ 810 
Q 597-18  T 579  ☑ 579 
Q 947-92  T 855  ☑ 855 
Q 897-560 T 337  ☑ 337 
Q 821-79  T 742  ☑ 742 
Q 774-20  T 754  ☑ 754 

--------------------------------------------------
Iteration 72
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 4s 200us/step - loss: 0.0839 - acc: 0.9739 - val_loss: 0.0856 - val_acc: 0.9692
Q 197-69  T 128  ☑ 128 
Q 846-2   T 844  ☑ 844 
Q 78-62   T 16   ☑ 16  
Q 995-365 T 630  ☒ 620 
Q 159-71  T 88   ☒ 87  
Q 648-273 T 375  ☑ 375 
Q 736-146 T 590  ☑ 590 
Q 528-33  T 495  ☑ 495 
Q 468-44  T 424  ☑ 424 
Q 949-512 T 437  ☑ 437 

--------------------------------------------------
Iteration 85
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 4s 208us/step - loss: 0.0426 - acc: 0.9911 - val_loss: 0.0660 - val_acc: 0.9791
Q 338-187 T 151  ☒ 141 
Q 705-81  T 624  ☑ 624 
Q 78-62   T 16   ☑ 16  
Q 107-28  T 79   ☑ 79  
Q 56-12   T 44   ☑ 44  
Q 842-39  T 803  ☑ 803 
Q 512-92  T 420  ☒ 410 
Q 556-8   T 548  ☑ 548 
Q 518-44  T 474  ☑ 474 
Q 777-51  T 726  ☑ 726 

--------------------------------------------------
Iteration 86
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 4s 199us/step - loss: 0.1037 - acc: 0.9648 - val_loss: 0.0906 - val_acc: 0.9705
Q 530-210 T 320  ☑ 320 
Q 473-33  T 440  ☑ 440 
Q 841-750 T 91   ☑ 91  
Q 633-33  T 600  ☑ 600 
Q 657-47  T 610  ☑ 610 
Q 994-167 T 827  ☑ 827 
Q 771-0   T 771  ☑ 771 
Q 948-71  T 877  ☑ 877 
Q 768-646 T 122  ☑ 122 
Q 619-74  T 545  ☑ 545 

--------------------------------------------------
Iteration 99
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 4s 200us/step - loss: 0.0353 - acc: 0.9925 - val_loss: 0.0525 - val_acc: 0.9845
Q 526-399 T 127  ☑ 127 
Q 785-666 T 119  ☑ 119 
Q 573-562 T 11   ☑ 11  
Q 524-0   T 524  ☑ 524 
Q 245-1   T 244  ☑ 244 
Q 578-78  T 500  ☑ 500 
Q 981-61  T 920  ☑ 920 
Q 317-285 T 32   ☑ 32  
Q 256-9   T 247  ☑ 247 
Q 139-5   T 134  ☑ 134 


### Testing

In [16]:
evaluated_loss, evaludated_accuracy = model.evaluate(x=test_x, y=test_y)

60000/60000 [==============================] - 20s 328us/step


In [17]:
print("testing loss:{}, testing accuracy:{}".format(evaluated_loss, evaludated_accuracy))

testing loss:0.06018444219728311, testing accuracy:0.9805291666666667
